In [17]:
import pandas as pd
import numpy as np

import data
import svm

In [18]:
#daily = data.get_transit_daily()
# gets a smaller version for faster troubleshooting
#daily = data.data_transforms(daily)
daily = data.get_mega_stop()

In [19]:
daily = daily.drop(columns=['apc_status', 'service_date', 'service_key'])
print(daily.dtypes)

arrive_time                           int64
data_source                           int64
direction                          category
door                               category
dwell                                 int64
estimated_load                        int64
leave_time                            int64
lift                               category
location_id                        category
maximum_speed                         int64
offs                                  int64
ons                                   int64
pattern_distance                    float32
route_number                       category
schedule_status                       int64
stop_time                             int64
train                              category
train_mileage                       float32
trip_number                           int64
vehicle_number                     category
x_coordinate                        float32
y_coordinate                        float32
day_of_year                     

In [20]:
daily['label'] = daily.groupby('trip_number')['arrival_deviance'].shift(-1)
daily = daily[~daily.isin([np.nan, np.inf, -np.inf]).any(1)]

In [21]:
x = daily
x = pd.get_dummies(x)
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

In [22]:
labels = x.pop("label")
print(labels)

202727    90.0
330011    79.0
330010    65.0
330009    55.0
330008    41.0
          ... 
426457   -47.0
426458   -34.0
426459   -27.0
426460   -52.0
426461   -46.0
Name: label, Length: 9998, dtype: float64


In [47]:
from sklearn.decomposition import PCA
# n components
pca = PCA(300)

In [48]:
print(x_scaled)
print(x.shape)

[[0.5945322  1.         0.         ... 0.         0.         0.        ]
 [0.66123937 1.         0.         ... 0.         0.         0.        ]
 [0.65953827 1.         0.         ... 0.         0.         0.        ]
 ...
 [0.15164034 1.         0.         ... 0.         0.         0.        ]
 [0.15419198 1.         0.03410853 ... 0.         0.         0.        ]
 [0.16075334 1.         0.         ... 0.         0.         0.        ]]
(9998, 2080)


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_scaled, labels)

proj = pca.fit_transform(X_train)
test_proj = pca.transform(X_test)
print("sum of variance_ratio: ", np.sum(pca.explained_variance_ratio_))
print("shape of new data: ", proj.shape)

model = svm.train_svm(proj, y_train)

sum of variance_ratio:  0.9950400500952727
shape of new data:  (7498, 300)


In [50]:
results = model.predict(test_proj)

In [55]:
# plot predicted and actual
import util
util.plot(69,420,results,y_test)

NameError: name 'util' is not defined

In [52]:
# get accuracy
from sklearn import metrics

print("MAE: ", metrics.mean_absolute_error(y_test,results))

MAE:  31.871722311094963


In [53]:
print("MSE: ", metrics.mean_squared_error(y_test,results))

MSE:  2916.81033081188


In [54]:
print("R2: ", metrics.r2_score(y_test,results))

R2:  0.6891931107113365
